In [1]:
import pandas as pd
import datetime
s=pd.Timestamp.today().normalize()  

In [6]:
today=s.strftime("%Y-%m-%d")
#print(today)
days_28_period= s - datetime.timedelta(28)
days_difference= s - days_28_period
days_28_period=days_28_period.strftime("%Y-%m-%d")
#days_28_period
print(days_difference)

28 days 00:00:00


In [7]:
from pandas import json_normalize
import json
import matplotlib.pyplot as plt
#import requests module
import requests
from requests.auth import HTTPBasicAuth
page=1
sales_data=[]
while(1):
    response = requests.get('https://api.cin7.com/api/v1/SalesOrders?rows=250&where= createdDate<'+str(today)+'T00:00:00Z and createdDate>'+days_28_period+'T23:59:59Z&fields=createdDate,lineItems&page='+str(page),auth = HTTPBasicAuth('DiggsIncUS', '9901f725991a42c4b8ea12917e3c21d9'))
    if not response.json():
        break
    #print (response.json())
    page_data=response.json()
    #separarting each object from the the output array and then appending it sales_Data to have an array of salesrecord objects
    sales_data += page_data
    #end of a single page
    #print("-------------------------------------------------") 
    #increasing the page value to iterate through all pages and get the data
    page+=1 
#print(sales_data)                    


In [8]:
sales_data

[{'createdDate': '2023-04-04T23:54:05Z',
  'lineItems': [{'id': 200280,
    'createdDate': '2023-04-04T23:54:05Z',
    'transactionId': 88826,
    'parentId': 0,
    'productId': 204,
    'productOptionId': 27,
    'integrationRef': '0',
    'sort': 10,
    'code': 'CRA002AAMG-R',
    'name': 'Refurbished Revol Dog Crate - M / Grey',
    'option1': 'Grey',
    'option2': 'M',
    'option3': '',
    'qty': 1.0,
    'styleCode': 'CRA002AAMG-R',
    'barcode': '',
    'sizeCodes': None,
    'lineComments': None,
    'unitCost': 221.64,
    'unitPrice': 285.0,
    'discount': 0.0,
    'qtyShipped': 1.0,
    'holdingQty': 0.0,
    'accountCode': '',
    'stockControl': 'FIFO',
    'stockMovements': [{'batch': None, 'quantity': 1.0, 'serial': None}]}]},
 {'createdDate': '2023-04-04T23:54:03Z',
  'lineItems': [{'id': 200279,
    'createdDate': '2023-04-04T23:54:03Z',
    'transactionId': 88825,
    'parentId': 0,
    'productId': 170,
    'productOptionId': 103,
    'integrationRef': '0',
   

In [9]:
df_sales = json_normalize(sales_data)
df=df_sales.explode('lineItems').reset_index(drop=True)
df=df.dropna().reset_index(drop=True)
df['code']=''
df['qty']=''
for i in range(len(df)):
    df.loc[i, "code"]=df.loc[i, "lineItems"]['code']
    df.loc[i, "qty"]=int(df.loc[i, "lineItems"]['qty'])
df=df.filter(['createdDate','code','qty'], axis=1)
df

,createdDate,code,qty
0,2023-04-04T23:54:05Z,CRA002AAMG-R,1
1,2023-04-04T23:54:03Z,PAD023AAMC,1
2,2023-04-04T23:49:56Z,GRO011AASTR,1
3,2023-04-04T23:31:08Z,PAD020AAMG,1
4,2023-04-04T22:36:47Z,CRA012AASW,1
...,...,...,...
7529,2023-03-09T00:29:00Z,CW0S5000V05G,1
7530,2023-03-09T00:20:00Z,CRA007AASB,1
7531,2023-03-09T00:20:00Z,PAD021AASC,1
7532,2023-03-09T00:20:00Z,GRO006AASSR,1


In [55]:
#accessing the skumapping excel
compare_df=pd.read_excel('/Users/nidhimenon/Downloads/DiggsSKUMapping.xlsx')
compare_df=compare_df.filter(['Diggs SKU', 'Variant'])
compare_df=compare_df.rename(columns={'Diggs SKU':'code'})
compare_df

,code,Variant
0,WSC007AALW,Collar - Large - Ash
1,WSC012AALP,Collar - Large - Blush
2,WSC040AALP,Collar - Large - Blush
3,WSC008AALB,Collar - Large - Charcoal
4,WSC016AALL,Collar - Large - Lilac
...,...,...
218,PAD008AAMT,V1 Snooz - Medium - Tan
219,PAD002AASB,V1 Snooz - Small - Dark Grey
220,PAD005AASC,V1 Snooz - Small - Dark Grey
221,PAD001AASG,V1 Snooz - Small - Light Grey


In [56]:
#matching the data from sales to the skumapping excel sheet to get product variants and their quantity sold
final_df=pd.merge(df, compare_df, on='code', how='left')
print(final_df)

               createdDate          code qty  \
0     2023-04-04T23:54:05Z  CRA002AAMG-R   1   
1     2023-04-04T23:54:03Z    PAD023AAMC   1   
2     2023-04-04T23:49:56Z   GRO011AASTR   1   
3     2023-04-04T23:31:08Z    PAD020AAMG   1   
4     2023-04-04T22:36:47Z    CRA012AASW   1   
...                    ...           ...  ..   
7529  2023-03-09T00:29:00Z  CW0S5000V05G   1   
7530  2023-03-09T00:20:00Z    CRA007AASB   1   
7531  2023-03-09T00:20:00Z    PAD021AASC   1   
7532  2023-03-09T00:20:00Z   GRO006AASSR   1   
7533  2023-03-09T00:01:52Z    PAD023AAMC   1   

                                Variant  
0     Refurbished Revol - Medium - Grey  
1            Snooz - Medium - Dark Grey  
2                     Groov - Turquoise  
3           Snooz - Medium - Light Grey  
4                   Revol - Small - Ash  
...                                 ...  
7529                                NaN  
7530           Revol - Small - Charcoal  
7531          Snooz - Small - Dark Grey  
753

In [57]:
#final_df.to_excel("/Users/nidhimenon/Desktop/output_nan.xlsx")
final_df=final_df.groupby("Variant")["qty"].sum()
#final_df=final_df.sort_values(ascending=False)
print(final_df)

Variant
Collar - Large - Ash                      1
Collar - Large - Blush                    1
Collar - Large - Charcoal                 6
Collar - Large - Sage                     2
Collar - Large - Slate                    1
                                       ... 
Snooz - Small - Light Grey              200
Spreadable Treat - Gold N Delicious    8598
Spreadable Treat - Thats My Jam         217
V1 Snooz - Small - Dark Grey             13
V1 Snooz - Small - Light Grey             6
Name: qty, Length: 133, dtype: int64


In [58]:
#final_df=(final_df/28)

In [59]:
final_df

Variant
Collar - Large - Ash                      1
Collar - Large - Blush                    1
Collar - Large - Charcoal                 6
Collar - Large - Sage                     2
Collar - Large - Slate                    1
                                       ... 
Snooz - Small - Light Grey              200
Spreadable Treat - Gold N Delicious    8598
Spreadable Treat - Thats My Jam         217
V1 Snooz - Small - Dark Grey             13
V1 Snooz - Small - Light Grey             6
Name: qty, Length: 133, dtype: int64

In [60]:
#accessing the inventory excel
inventory_df=pd.read_excel('/Users/nidhimenon/Desktop/output_summary.xlsx')
inventory_df=inventory_df.iloc[: , 1:]
invent_df=inventory_df.groupby("Variant")["stockAvailable"].sum()
invent_df

Variant
Collar - Large - Ash              4104
Collar - Large - Blush             950
Collar - Large - Charcoal         1490
Collar - Large - Lilac            1365
Collar - Large - Navy             4080
                                  ... 
V1 Snooz - Medium - Dark Grey        0
V1 Snooz - Medium - Light Grey       0
V1 Snooz - Medium - Tan              0
V1 Snooz - Small - Dark Grey         3
V1 Snooz - Small - Light Grey        0
Name: stockAvailable, Length: 138, dtype: int64

In [61]:
predict_df= pd.merge(final_df, inventory_df, on='Variant')

In [62]:
predict_df.rename(columns={'qty_x':'avg_sales'})

,Variant,avg_sales,Product Title,stockAvailable,createdDate,qty_y,Sell_through_rate
0,Collar - Large - Ash,1,Collar,1368,2023-01-13,2,0.146199
1,Collar - Large - Ash,1,Collar,1368,2023-01-22,1,0.073099
2,Collar - Large - Ash,1,Collar,1368,2023-01-25,1,0.073099
3,Collar - Large - Blush,1,Collar,475,2023-01-12,1,0.210526
4,Collar - Large - Blush,1,Collar,475,2023-01-14,1,0.210526
...,...,...,...,...,...,...,...
1973,V1 Snooz - Small - Light Grey,6,V1 Snooz,0,2023-01-26,2,0.000000
1974,V1 Snooz - Small - Light Grey,6,V1 Snooz,0,2023-01-27,2,0.000000
1975,V1 Snooz - Small - Light Grey,6,V1 Snooz,0,2023-01-28,3,0.000000
1976,V1 Snooz - Small - Light Grey,6,V1 Snooz,0,2023-01-29,1,0.000000


In [ ]:
predict_df['month_time']=df[]